In [1]:
import os

import langchain
from langchain.tools import tool, StructuredTool

print(f"Running with langchain version {langchain.__version__}")
from dotenv import load_dotenv

load_dotenv()

chatgptkey = os.getenv("CHATGPT_API_KEY")

Running with langchain version 0.3.1


In [8]:
@tool
def simple_test_tool(who: str, times: str) -> str:
    """Just a simple test tool, takes a string representing a name and returns a greeting"""
    t = int(times)
    return f"{who}, hello from the simple test tool, {t} times"

simple_test_tool.invoke(input={'who':"world", 'times':"3"})

'world, hello from the simple test tool, 3 times'

In [12]:

def try_structuredtool_simple():
    
    


In [13]:
from typing import Awaitable


class TestToolChest:
    
    def __init__(self, name: str):
        self.name = name
    
    def greet(self, who: str) -> str:
        return f"Hello, {who}, from {self.name}!"
    
    def greet_async(self, who: str) -> Awaitable[str]:
        return f"Hello, {who}, from {self.name}!"
    
    def greet_with_times(self, who: str, times: int) -> str:
        return f"Hello, {who}, from {self.name}! " * times
    
    def greet_with_times_async(self, who: str, times: int) -> Awaitable[str]:
        return f"Hello, {who}, from {self.name}! " * times
    


ValidationError: 2 validation errors for MethodTool
instance
  Field required [type=missing, input_value={'func': <bound method Te....<locals>.DummySchema'>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
method_name
  Field required [type=missing, input_value={'func': <bound method Te....<locals>.DummySchema'>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [25]:
from dataclasses import dataclass, field
from typing import Callable, Any, Awaitable


@dataclass
class MethodTool:
    """A tool that wraps a method of a class instance."""
    
    instance: Any
    method_name: str
    func: Callable = field(init=False)
    
    def __post_init__(self):
        # Get the method from the instance
        self.func = getattr(self.instance, self.method_name)

    def invoke(self, input: dict) -> Any:
        # Unpack input and call the method
        return self.func(**input)


class TestToolChest:
    
    def __init__(self, name: str):
        self.name = name
    
    def greet(self, who: str) -> str:
        """Greet a person."""
        return f"Hello, {who}, from {self.name}!"
    
    def greet_async(self, who: str) -> Awaitable[str]:
        """Greet a person asynchronously."""
        return f"Hello, {who}, from {self.name}!"
    
    def greet_with_times(self, who: str, times: int) -> str:
        """Greet a person multiple times."""
        return f"Hello, {who}, from {self.name}! " * times
    
    def greet_with_times_async(self, who: str, times: int) -> Awaitable[str]:
        """Greet a person multiple times asynchronously."""
        return f"Hello, {who}, from {self.name}! " * times


# Create an instance of TestToolChest
ttc1 = TestToolChest("ttc1")

# Create a tool from the greet method
greet_tool = MethodTool(instance=ttc1, method_name="greet")

# Invoke the tool
result = greet_tool.invoke(input={'who': "world"})
print(result)


Hello, world, from ttc1!


In [26]:
ttc1

In [27]:
import functools


In [28]:
myfn = functools.partial(ttc1.greet, self = ttc1)

In [29]:
myfn("world")

TypeError: TestToolChest.greet() got multiple values for argument 'self'

In [31]:
myfn = ttc1.greet



In [36]:
myfn("world")

'Hello, world, from New name for ttc1!'

In [33]:
testtool = StructuredTool.from_function(ttc1.greet)

In [34]:
testtool.invoke(input={'who': "world"})

'Hello, world, from ttc1!'

In [35]:
ttc1.name = "New name for ttc1"


In [37]:
import os
import langchain
from langchain_core.prompts import ChatPromptTemplate
from typing_extensions import TypedDict
from langchain.tools import StructuredTool
print(f"Running with langchain version {langchain.__version__}")
from langchain.chat_models import init_chat_model 
from dotenv import load_dotenv

from buildwithai.langchain_tools import get_summarized_jira_issue, get_ticket_description

load_dotenv()

chatgptkey = os.getenv("CHATGPT_API_KEY")

Running with langchain version 0.3.1


In [60]:
class TestInput(TypedDict):
    """Provide the correct values to the function. THe function must take parameters by these exact names.
    However it is not entirely necessary to do it like this, you can get the structured output using the tool or the underlying function as below in chain and chain3."""
    
    who: str
    
llm = init_chat_model("gpt-4o-mini", model_provider="openai", api_key = chatgptkey)
findthename = "Extract the name from the input after analyzing it carefully."
theprompt = ChatPromptTemplate.from_messages([("system", findthename), ("human", "{input}")])
testtool = StructuredTool.from_function(ttc1.greet)  #This will build the tool from the method greet in the TestToolChest class.


chain1 = theprompt | llm.with_structured_output(TestInput) | testtool
chain = theprompt | llm.with_structured_output(testtool) | testtool  #In example code this was given as a class, like TestInput above, but this seems to work fine, and its' param names are always going to match!
# I think that perhaps the __or__ override for the Tool class should be changed to do this automatically, so that the user doesn't have to set up an llm.with_structured_output() call manually.???
chain3 = theprompt | llm.with_structured_output(ttc1.greet) | StructuredTool.from_function(ttc1.greet)  #This works too, and it will build the tool from the method greet in the TestToolChest class.

chain.invoke("Is my name Joe? I have seen many names, such as Peter, Paul, Lucy, and John, but others have always called me Larry, even though their names are like Mary and Catherine.")


#Note, it might also be possible to use bind on the  Runnable to set the input type, but I'm not sure how that would work with the structured output.

'Hello, Larry, from Wellingdone!'

In [49]:
ttc1.name = "Wellingdone"

In [50]:
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [58]:
chain1.invoke(" I have seen many names, such as Peter, Paul, Lucy, and John, but others have always called me by the long form of the name Bob which rhymes with Dessert, even though their names are like Mary and Catherine.")

'Hello, Robert, from Wellingdone!'

In [62]:
res = chain3.invoke(" I have seen many names, such as Peter, Paul, Lucy, and John, but others have always called me by the long form of the name Bob which rhymes with Dessert, even though their names are like Mary and Catherine.")

In [63]:
type(res)

str

In [64]:
res


'Hello, Robert, from Wellingdone!'

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

router_promp = ("Given a ticket id, find out what type of ticket it is, and if it is an epic or an issue that has subtasks, build a tree of the full ticket hierachy. Explain"
                "who is responsible for the ticket, and what the ticket is about. If the ticket is a subtask, explain what the parent ticket is about, and who is responsible for it.")
router_prompt = ChatPromptTemplate.from_template(router_promp)
router = LLMRouterChain("gpt-4o-mini", model_provider="openai", api_key = chatgptkey)
router.add_route("get_ticket_description", get_ticket_description)
router.add_route("get_summarized_jira_issue", get_summarized_jira_issue)

rop = RouterOutputParser()
router_chain = router_prompt | router | rop
